In [33]:
from pyspark.sql import SparkSession
import pyspark.sql.functions as fc

In [8]:
spark = SparkSession.builder.config("spark.jars.packages","com.johnsnowlabs.nlp:spark-nlp_2.11:2.4.4").getOrCreate()
spark

# Netflix Catalog

### Load catalog data and get ride of malformed rows

In [116]:
netflix_catalog_path='/Users/brayanjules/Projects/personal/data engineer/datasets'
catalog = spark.read.csv(netflix_catalog_path,inferSchema=True, header=True,mode="DROPMALFORMED")

In [117]:
catalog.printSchema()

root
 |-- show_id: string (nullable = true)
 |-- type: string (nullable = true)
 |-- title: string (nullable = true)
 |-- director: string (nullable = true)
 |-- cast: string (nullable = true)
 |-- country: string (nullable = true)
 |-- date_added: string (nullable = true)
 |-- release_year: string (nullable = true)
 |-- rating: string (nullable = true)
 |-- duration: string (nullable = true)
 |-- listed_in: string (nullable = true)
 |-- description: string (nullable = true)



In [138]:
catalog_ordered=catalog.orderBy(desc('title'))
catalog_ordered.limit(3).toPandas()

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description
0,81011096,Movie,최강전사 미니특공대 : 영웅의 탄생,Young Jun Lee,"Um Sang-hyun, Yang Jeong-hwa, Jeon Tae-yeol, S...",None,"September 1, 2018",2018,TV-Y7-FV,68 min,Children & Family Movies,"Miniforce, a special task force of elite range..."
1,80226357,Movie,반드시 잡는다,Hong-seon Kim,Baek Yoon-sik,South Korea,"February 28, 2018",2017,TV-MA,110 min,"Dramas, International Movies, Thrillers",After people in his town start turning up dead...
2,80226338,TV Show,마녀사냥,None,"Si-kyung Sung, Se-yoon Yoo, Dong-yup Shin, Ji-...",South Korea,"February 19, 2018",2015,TV-MA,1 Season,"International TV Shows, Korean TV Shows, Stand...",Four Korean celebrity men and guest stars of b...


## Netflix Catalog Data Cleaning

### Deduplication of the data

In [98]:
duplicatedContent=catalog.groupBy(['title','director']).count().orderBy(fc.desc('count'))
duplicatedContent.limit(5).toPandas()

,title,director,count
0,None,None,6725
1,Life,None,2
2,The Silence,Gajendra Ahire,2
3,The Birth Reborn,Eduardo Chauvet,2
4,Top Boy,None,2


In [104]:
catalog.where(catalog.title=='The Silence').dropDuplicates(['title','director']).toPandas()

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description
0,81021447,Movie,The Silence,John R. Leonetti,"Stanley Tucci, Kiernan Shipka, Miranda Otto, K...",Germany,"April 10, 2019",2019,TV-14,91 min,"Horror Movies, Thrillers",With the world under attack by deadly creature...
1,80244078,Movie,The Silence,Gajendra Ahire,"Raghuvir Yadav, Nagraj Manjule, Anjali Patil, ...",India,"March 1, 2018",2017,TV-MA,90 min,"Dramas, International Movies","On a train in Mumbai, 20-something Chini witne..."


In [119]:
catalog.dropDuplicates(['title','director']).groupBy(['title','director']).count().orderBy(fc.desc('count')).toPandas()

,title,director,count
0,I'll See You in My Dreams,Brett Haley,1
1,Raging Bull,Martin Scorsese,1
2,Barbie: Spy Squad,Conrad Helten,1
3,Drive,Tarun Mansukhani,1
4,Yaara O Dildaara,Ksshitij Chaudhary,1
...,...,...,...
7096,Lunatics,None,1
7097,Real Rob,None,1
7098,Explained,None,1
7099,Castlevania,None,1


In [120]:
no_valid_content=catalog.where((catalog.title).isNotNull() & (catalog.description).isNull())
no_valid_content.limit(5).toPandas()

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description


In [121]:
non_duplicated_content=catalog.dropDuplicates(['title','director']).orderBy(fc.desc('title'))
non_duplicated_content.limit(3).toPandas()

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description
0,81011096,Movie,최강전사 미니특공대 : 영웅의 탄생,Young Jun Lee,"Um Sang-hyun, Yang Jeong-hwa, Jeon Tae-yeol, S...",None,"September 1, 2018",2018,TV-Y7-FV,68 min,Children & Family Movies,"Miniforce, a special task force of elite range..."
1,80226357,Movie,반드시 잡는다,Hong-seon Kim,Baek Yoon-sik,South Korea,"February 28, 2018",2017,TV-MA,110 min,"Dramas, International Movies, Thrillers",After people in his town start turning up dead...
2,80226338,TV Show,마녀사냥,None,"Si-kyung Sung, Se-yoon Yoo, Dong-yup Shin, Ji-...",South Korea,"February 19, 2018",2015,TV-MA,1 Season,"International TV Shows, Korean TV Shows, Stand...",Four Korean celebrity men and guest stars of b...


### Drop rows without title.

In [255]:
content_with_title=non_duplicated_content.dropna('any',subset=['title']).orderBy(fc.asc('title'))
content_with_title=content_with_title.withColumn('title',fc.translate('title','"',''))
content_with_title.limit(3).toPandas()

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description
0,80132127,Movie,Behind The Cove: The Quiet Japanese Speak Out,Keiko Yagi,None,"Japan, United States","August 25, 2017",2015,TV-14,105 min,"Documentaries, International Movies",After a documentary about the Japanese whaling...
1,81168345,Movie,Escape from the Liberty Cinema,Wojciech Marczewski,"Janusz Gajos, Zbigniew Zamachowski, Teresa Mar...",Poland,"October 1, 2019",1990,TV-MA,88 min,"Comedies, Dramas, Independent Movies",Artistic rebellion ignites at the movies when ...
2,81087095,Movie,#Roxy,Michael Kennedy,"Jake Short, Sarah Fisher, Booboo Stewart, Dann...",Canada,"April 10, 2019",2018,TV-14,105 min,"Comedies, Romantic Movies",A teenage hacker with a huge nose helps a cool...


In [134]:
topDirector=content_with_title.dropna('any',subset=['director']).where((catalog.type=='Movie')).groupBy('director').count().orderBy(fc.desc('count'))
topDirector.limit(3).toPandas()

,director,count
0,"Raúl Campos, Jan Suter",18
1,Marcus Raboy,14
2,Jay Karas,13


In [135]:
content_with_title.count()

7096

# Reddit Netflix comments

In [454]:
import praw
import os
import pandas as pd
import pyspark.sql.types as tp
from pyspark.sql import Row

In [453]:
os.getcwd()

'/Users/brayanjules/Projects/personal/data engineer/nanodegre/capstone-project'

In [458]:
reddit = praw.Reddit(site_name='default',
                     user_agent='academic_comments_understanding:v1 by /u/zekeja') ## Use the praw.init when possible

NoSectionError: No section: 'default'
You provided the name of a praw.ini configuration which does not exist.

For help with creating a Reddit instance, visit
https://praw.readthedocs.io/en/latest/code_overview/reddit_instance.html

For help on configuring PRAW, visit
https://praw.readthedocs.io/en/latest/getting_started/configuration.html

### SubReddit Search by content ( netflix,NetflixBestOf,bestofnetflix)

In [431]:
redditSchema=tp.StructType([tp.StructField('show_id',tp.StringType(),True),
               tp.StructField('submission_id',tp.StringType(),True),
               tp.StructField('source',tp.StringType(),True),
               tp.StructField('title',tp.StringType(),True),
               tp.StructField('description',tp.StringType(),True),
               tp.StructField('created_utc',tp.StringType(),True),
               tp.StructField('author',tp.StringType(),True),
               tp.StructField('score',tp.StringType(),True),
               tp.StructField('spoiler',tp.StringType(),True),
               tp.StructField('is_original_content',tp.StringType(),True),
               tp.StructField('distinguished',tp.StringType(),True),
               tp.StructField('link',tp.StringType(),True),             
               tp.StructField('comments',tp.ArrayType(tp.StructType([
                   tp.StructField('comment_id',tp.StringType(),True),
                   tp.StructField('body',tp.StringType(),True),
                   tp.StructField('created_utc',tp.StringType(),True),
                   tp.StructField('score',tp.StringType(),True),
                   tp.StructField('parent_id',tp.StringType(),True),
                   tp.StructField('submission_id',tp.StringType(),True)]
               )),True)
              ])


In [442]:
content_rows=[]
for content in content_with_title.limit(100).collect():
    title_split=content.title.split(":",1)
    content_title=title_split[0]
    subreddit=reddit.subreddit('netflix')
    for sm in subreddit.search('"'+content_title+'"',sort='new'):
        sm.comments.replace_more(limit=None)
        #print(sm.title)
        row_comments = []
        for comment in sm.comments.list():
            row_comments.append((comment.id,comment.body,comment.created_utc,comment.score,
                                       comment.parent_id,comment.link_id)) 
        current_sm=(content.show_id,sm.id,subreddit.display_name,sm.title,sm.selftext,sm.created_utc,sm.author.name,
                      sm.score,sm.spoiler,sm.is_original_content,sm.distinguished,sm.permalink,row_comments)
        content_rows.append(current_sm)    

In [443]:
rdt_netflix_content=spark.createDataFrame(content_rows,redditSchema)
rdt_netflix_content.printSchema()

root
 |-- show_id: string (nullable = true)
 |-- submission_id: string (nullable = true)
 |-- source: string (nullable = true)
 |-- title: string (nullable = true)
 |-- description: string (nullable = true)
 |-- created_utc: string (nullable = true)
 |-- author: string (nullable = true)
 |-- score: string (nullable = true)
 |-- spoiler: string (nullable = true)
 |-- is_original_content: string (nullable = true)
 |-- distinguished: string (nullable = true)
 |-- link: string (nullable = true)
 |-- comments: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- comment_id: string (nullable = true)
 |    |    |-- body: string (nullable = true)
 |    |    |-- created_utc: string (nullable = true)
 |    |    |-- score: string (nullable = true)
 |    |    |-- parent_id: string (nullable = true)
 |    |    |-- submission_id: string (nullable = true)



In [447]:
rdt_netflix_content.limit(5).toPandas()

,show_id,submission_id,source,title,description,created_utc,author,score,spoiler,is_original_content,distinguished,link,comments
0,80132127,98y3ld,netflix,"[UK] Netflix no longer has The Cove, the Oscar...",I appreciate having both sides of the argument...,1.534803751E9,lewis_pritchard,143,false,false,None,/r/netflix/comments/98y3ld/uk_netflix_no_longe...,"[(e4jp9zf, It's highly unlikely they know the ..."
1,81092768,e8v216,netflix,Weird shirtless gym selfie on Netflix Twitter ...,[https://twitter.com/netflix/status/120445343...,1.576004863E9,OrganicCorndawg,1,false,false,None,/r/netflix/comments/e8v216/weird_shirtless_gym...,"[(faeoqgb, This response seems to explain it:\..."
2,81092768,99vbkh,netflix,Selfie From Hell Movie: Ending Explained + Wha...,,1.53509487E9,PaulTweddle,5,false,false,None,/r/netflix/comments/99vbkh/selfie_from_hell_mo...,[]
3,80125979,8z4t4l,netflix,what the FUCK,was at a party right now (we all bet on croati...,1.531687004E9,dickensian_nightmare,0,false,false,None,/r/netflix/comments/8z4t4l/what_the_fuck/,"[(e2g8bs1, I’ll Give you an upvote because of ..."
4,80125979,6fhaki,netflix,"List of All Original Netflix Movies with RT, M...","When I started this, it was to see how popular...",1.496698381E9,arbn17,43,false,false,None,/r/netflix/comments/6fhaki/list_of_all_origina...,"[(diiccw7, War Machine was a war-comedy? I fa..."


In [451]:
len(rdt_netflix_content.toPandas())

1684

In [448]:
rdt_netflix_content.write.json("/Users/brayanjules/Projects/personal/data engineer/datasets/reddit_netflix",'overwrite')
